In [71]:
# @title Magyar Irodalomtörténet Bibliográfiája rekord ellenőrző {display-mode: "form"}

# @title Földrajzi nevek keresése {display-mode: "form"}

import requests

# Define the Wikidata API endpoint
api_endpoint = "https://itidata.abtk.hu/w/api.php"

# Define the Wikidata Query Service endpoint
wdqs_endpoint = "https://query.itidata.abtk.hu/proxy/wdqs/bigdata/namespace/wdq/sparql"

# Set the request headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0",
    "Accept": "application/json"
}

def get_itidata_item_id(label):
    # Set the request parameters
    params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": "hu",
        "search": label,
        "type": "item",
    }

    try:
        # Send the HTTP request to the itidata API
        response = requests.get(api_endpoint, params=params)
        data = response.json()

        # Check if any entities were found
        if data.get("search"):
            # Get a list of itidata item IDs from the search results
            itidata_item_ids = [result["id"] for result in data["search"]]

            return itidata_item_ids

    except Exception as e:
        print(f"Error: {e}")

    # Return an empty string if an error occurred or no match found
    return ""

def get_label_json_property_itidata(item_id, what_do_yo_need):

    # Set the request parameters
    params = {
        "action": "wbgetentities",
        "ids": item_id,
        "languages": "hu|en",  # Specify languages for labels (Hungarian and English)
        "format": "json"
    }

    try:
        # Send the HTTP request to the Wikidata API
        response = requests.get(api_endpoint, params=params)
        data = response.json()

        # Extract relevant information from the response
        entity = data.get("entities", {}).get(item_id, {})
        labels = entity.get("labels", {})
        label_hu = labels.get("hu", {}).get("value", "")
        label_en = labels.get("en", {}).get("value", "")

        if what_do_yo_need == "json":
            return data
        elif what_do_yo_need.lstrip("P").isdigit():
            property_value = data["entities"][item_id]["claims"][what_do_yo_need][0]["mainsnak"]["datavalue"]["value"]["id"]
            return property_value
        else:
            return (label_hu, label_en)

    except Exception as e:
        print(f"Error: {e}")

    # Return empty strings if an error occurred
    return "Unknown"

def get_item_labels_from_itidata(item_id):

    # Set the request parameters
    params = {
        "action": "wbgetentities",
        "ids": item_id,
        "languages": "hu|en",  # Specify languages for labels (Hungarian and English)
        "format": "json"
    }

    try:
        # Send the HTTP request to the Wikidata API
        response = requests.get(api_endpoint, params=params)
        data = response.json()

        # Extract relevant information from the response
        entity = data.get("entities", {}).get(item_id, {})
        labels = entity.get("labels", {})
        label_hu = labels.get("hu", {}).get("value", "")
        label_en = labels.get("en", {}).get("value", "")

        if label_hu is None or label_hu == "":
            return label_en + f"({item_id})"
        else:
            return label_hu + f"({item_id})"

    except Exception as e:
        print(f"Error: {e}")

    # Return empty strings if an error occurred
    return "Unknown"


# print(get_itidata_subclasses_of_human_settlement())
# print(get_item_labels_from_itidata("Q2727"))
# bud_ids = get_itidata_item_id("Budapest")
# print(bud_ids)
# bud_dict = get_itidata_item_info(bud_ids)
# for key, value in bud_dict.items():
#     print(f"{key}: {value}")

mib_instance_of_dict = {"Q24": "szakcikk", "Q23": "cikk", "Q24104": "szakkönyv", "Q17": "könyv"}
periodicals_dict = {"Q18": "folyóirat", "Q19": "irodalmi lap", "Q308137": "hírlap", "Q20": "tudományos folyóirat", "Q22": "újság"}
itidata_input_id = ""

# Input itidata id 
while itidata_input_id.lstrip("Q").isdigit() is False:
    itidata_input_id = input("ITIdata entitás id [Q12345]: ")
itidata_json = get_label_json_property_itidata(itidata_input_id, "json")

quality_check = True

# Check Englis and Hungarian lables
try:
    if itidata_json["entities"][itidata_input_id]["labels"]["hu"]["value"] == "":
        print("\033[91mHiányzik a magyar nyelvű címke.\033[0m")
        quality_check = False
    else:
        print("Magyar címke: ", itidata_json["entities"][itidata_input_id]["labels"]["hu"]["value"])
    if itidata_json["entities"][itidata_input_id]["labels"]["en"]["value"] == "":
        print("\033[91mHiányzik a magyar nyelvű címke.\033[0m")
        quality_check = False
    else:
        print("Angol címke: ", itidata_json["entities"][itidata_input_id]["labels"]["en"]["value"])
except KeyError:
    print("\033[91mHiányzó címke.\033[0m")
    quality_check = False

# Chech P1
try:
    instance_of = itidata_json["entities"][itidata_input_id]["claims"]["P1"][0]["mainsnak"]["datavalue"]["value"]["id"]
    if instance_of not in mib_instance_of_dict.keys():
        print(f"\033[91mA P1 értéke ({get_item_labels_from_itidata(instance_of)}) nem megengedett! Lehetséges értékek: \n"
          f"{[(key, value) for key, value in mib_instance_of_dict.items()]}\033[0m")
        quality_check = False
    else:
        print(f"Osztály, amelynek példánya: {mib_instance_of_dict[instance_of]} ({instance_of})")

        # Periodicals
        if instance_of == "Q23" or instance_of == "Q24":
           
            # Published in
            try:
                published_in = itidata_json["entities"][itidata_input_id]["claims"]["P44"][0]["mainsnak"]["datavalue"]["value"]["id"]
                print("Mű, amelyben megjelent (P44): ", get_item_labels_from_itidata(published_in))
                if get_label_json_property_itidata(published_in, "P1") not in periodicals_dict.keys():
                    print(f"\033[91mA mű, amnelyben megjelent (P44) nem periodika. \n"
                          f"Lehetséges értékek: {[(key, value) for key, value in periodicals_dict.items()]}\033[0m")
                    quality_check = False
            except KeyError:
                print("\033[91mHiányzik a mű, amelyben megjelent (P44) állítás.\033[0m")
                quality_check = False
           
           # Volume
            try:
                volume = itidata_json["entities"][itidata_input_id]["claims"]["P45"][0]["mainsnak"]["datavalue"]["value"]
                if volume.strip() == "":
                    print("\033[91mAz évfolyam, kötet (P45) értéke üres.\033[0m")
                    quality_check = False
                else:
                    print("Évfolyam, kötet (P45): ", volume)
            except KeyError:
                print("\033[91mHiányzik az évfolyam, kötet (P45).\033[0m")
                quality_check = False
                
            # Issue
            try:
                issue = itidata_json["entities"][itidata_input_id]["claims"]["P46"][0]["mainsnak"]["datavalue"]["value"]
                if issue.strip() == "":
                    print("\033[91mA szám (P46) értéke üres.\033[0m")
                    quality_check = False
                else:
                    print("Szám (P46): ", volume)
            except KeyError:
                print("\033[91mHiányzik a szám (P46).\033[0m")
                quality_check = False
            
except:
    print(f"\033[91mHiányzik az Osztály amelynek példánya (P1) értéke. Lehetséges értékek: \n"
          f"{[(key, value) for key, value in mib_instance_of_dict.items()]}\033[0m")
    quality_check = False

# Check MIB as collection
# TODO: What if the record is part of multiple collections?
try:
    collection = itidata_json["entities"][itidata_input_id]["claims"]["P17"][0]["mainsnak"]["datavalue"]["value"]["id"]
    if collection != "Q2138":
        print(f"\033[91mA Gyűjtemény (P17) értéke ({collection}) hibás. Helyes érték: A magyar irodalomtörténet bibliográfiája (Q2138).\033[0m")
        quality_check = False
except KeyError:
    print("\033[91mHiányzik: Gyűjtemény (P17): A magyar irodalomtörténet bibliográfiája (Q2138).\033[0m")
    quality_check = False

# Check genre
try: 
    genre = itidata_json["entities"][itidata_input_id]["claims"]["P41"][0]["mainsnak"]["datavalue"]["value"]["id"]
    print("Műfaj: ", get_item_labels_from_itidata(genre))
except KeyError:
    print("\033[91mHiányzik a műfaj (P41).\033[0m")
    quality_check = False

# Check author entity    
print(itidata_json["entities"][itidata_input_id]["claims"]["P7"])
try:
    author_id = itidata_json["entities"][itidata_input_id]["claims"]["P7"][0]["mainsnak"]["datavalue"]["value"]["id"]
    print("Szerző entitás: ", get_item_labels_from_itidata(author_id))
    if get_label_json_property_itidata(author_id, "P1") != "Q2":
        print("\033[91mA szerző entitás osztálya (P1) nem ember (Q2).\033[0m")
        quality_check = False
except KeyError:
    print("\033[91mHiányzó szerző entitás (P7).\033[0m")
    quality_check = False

# Publication date    
try:
    publication_date = itidata_json["entities"][itidata_input_id]["claims"]["P57"][0]["mainsnak"]["datavalue"]["value"]["time"]
    print("Megjelenés ideje (P57): ", publication_date)
except KeyError:
    print("\033[91mHiányzik vagy hibás a megjelenés ideje (P57).\033[0m")
    
if quality_check:
    print("Ellenőrző kód: ", hash(str(itidata_json)))
    
"""
# Q339115
[{'mainsnak': {'snaktype': 'somevalue', 'property': 'P7', 'hash': 'b9225ab78d331f81968611dd24b443b5573055da', 'datatype': 'wikibase-item'}, 'type': 'statement', 'qualifiers': {'P39': [{'snaktype': 'value', 'property': 'P39', 'hash': 'c954c9f829bf3153bbf8646ba8cf35fdded27051', 'datavalue': {'value': 'nincs', 'type': 'string'}, 'datatype': 'string'}]}, 'qualifiers-order': ['P39'], 'id': 'Q339115$2a67fdf1-4a7b-db7c-21f4-db47365c5739', 'rank': 'normal'}]
# Q25014
[{'mainsnak': {'snaktype': 'value', 'property': 'P7', 'hash': 'e0a1a464bf68b5febf375db9a09af1cf19e30df2', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 60760, 'id': 'Q60760'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id': 'Q25014$edbd253e-48b0-674a-11df-ff6347724bd8', 'rank': 'normal'}]
"""

Magyar címke:  Ilyenkor, szeptemberben…
Angol címke:  Ilyenkor, szeptemberben…
Osztály, amelynek példánya: cikk (Q23)
Mű, amelyben megjelent (P44):  Evangélikus Élet(Q23983)
Évfolyam, kötet (P45):  77.
Szám (P46):  77.
Műfaj:  önvallomás(Q24129)
[{'mainsnak': {'snaktype': 'value', 'property': 'P7', 'hash': 'e0a1a464bf68b5febf375db9a09af1cf19e30df2', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 60760, 'id': 'Q60760'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id': 'Q25014$edbd253e-48b0-674a-11df-ff6347724bd8', 'rank': 'normal'}]
Szerző entitás:  Fabiny Tamás(Q60760)
Hiányzik vagy hibás a megjelenés ideje (P57).
Ellenőrző kód:  2799894408070501538
